# Xception上的测试

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import numpy as np
from sklearn.utils import shuffle
import pandas as pd

import h5py
import os
import shutil

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## 数据处理
读取训练和测试图片，格式化成（299,299,3）的格式。并且划分训练集和验证集

In [2]:
import numpy as np
from tqdm import tqdm
import cv2

np.random.seed(2017)

n = 24962
X = np.zeros((n, 299, 299, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

train_cat2 = os.listdir("train2/cat")
train_dog2 = os.listdir("train2/dog")

i = 0
for filename in train_cat2:
    X[i] = cv2.resize(cv2.imread('train/%s' % filename), (299, 299))
    y[i] = 0
    i += 1
for filename in train_dog2:
    X[i] = cv2.resize(cv2.imread('train/%s' % filename), (299, 299))
    y[i] = 1
    i += 1

In [3]:
m = 12500
X_test = np.zeros((m, 299, 299, 3), dtype=np.uint8)

test_file = os.listdir("test")
j = 0
for filename in test_file:
    X_test[j] = cv2.resize(cv2.imread('test/%s' % filename), (299, 299))
    j += 1

In [4]:
print("i=", i)
print("X_test size:", X_test.shape)

i= 24962
X_test size: (12500, 299, 299, 3)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

## 模型一
去除顶层，锁定其他所有层，不参与训练；在这模型的基础上，加上GAP层和Dropout层，加sigmoid激活函数得出二分类的结果

In [6]:
base_model = Xception(input_tensor=Lambda(xception.preprocess_input)(Input((299, 299, 3))), weights='imagenet', include_top=False)

for layers in base_model.layers:
    layers.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(base_model.input, x)

In [7]:
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_valid, y_valid))

Train on 19969 samples, validate on 4993 samples
Epoch 1/5
19969/19969 [==============================] - 102s 5ms/step - loss: 0.1283 - acc: 0.9637 - val_loss: 0.0876 - val_acc: 0.9770
Epoch 2/5
19969/19969 [==============================] - 99s 5ms/step - loss: 0.0688 - acc: 0.9761 - val_loss: 0.1021 - val_acc: 0.9692
Epoch 3/5
19969/19969 [==============================] - 99s 5ms/step - loss: 0.0584 - acc: 0.9803 - val_loss: 0.1117 - val_acc: 0.9654
Epoch 4/5
19969/19969 [==============================] - 99s 5ms/step - loss: 0.0559 - acc: 0.9794 - val_loss: 0.0769 - val_acc: 0.9752
Epoch 5/5
19969/19969 [==============================] - 99s 5ms/step - loss: 0.0573 - acc: 0.9804 - val_loss: 0.0841 - val_acc: 0.9732


In [9]:
for i in range(len(model.layers)):
    print(i,model.layers[i].name)

0 input_1
1 lambda_1
2 block1_conv1
3 block1_conv1_bn
4 block1_conv1_act
5 block1_conv2
6 block1_conv2_bn
7 block1_conv2_act
8 block2_sepconv1
9 block2_sepconv1_bn
10 block2_sepconv2_act
11 block2_sepconv2
12 block2_sepconv2_bn
13 conv2d_1
14 block2_pool
15 batch_normalization_1
16 add_1
17 block3_sepconv1_act
18 block3_sepconv1
19 block3_sepconv1_bn
20 block3_sepconv2_act
21 block3_sepconv2
22 block3_sepconv2_bn
23 conv2d_2
24 block3_pool
25 batch_normalization_2
26 add_2
27 block4_sepconv1_act
28 block4_sepconv1
29 block4_sepconv1_bn
30 block4_sepconv2_act
31 block4_sepconv2
32 block4_sepconv2_bn
33 conv2d_3
34 block4_pool
35 batch_normalization_3
36 add_3
37 block5_sepconv1_act
38 block5_sepconv1
39 block5_sepconv1_bn
40 block5_sepconv2_act
41 block5_sepconv2
42 block5_sepconv2_bn
43 block5_sepconv3_act
44 block5_sepconv3
45 block5_sepconv3_bn
46 add_4
47 block6_sepconv1_act
48 block6_sepconv1
49 block6_sepconv1_bn
50 block6_sepconv2_act
51 block6_sepconv2
52 block6_sepconv2_bn
53 b

In [10]:
model.save_weights('xception_weights_1.h5')  
y_pred = model.predict(X_test, verbose=1)

12500/12500 [==============================] - 54s 4ms/step


In [11]:
y_pred = y_pred.clip(min=0.005, max=0.995)
df = pd.read_csv("sample_submission.csv")
for i, fname in enumerate(test_file):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('xception_predict_1.csv', index=None)
df.head(2)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,0.979796
1,2,0.995000


在kaggle上的得分是0.11558

## 模型二
fine-tuning 锁前70层，放开70层之后的层参与训练

In [12]:
for layers in base_model.layers:
    layers.trainable = False
for layer in model.layers[70:]:
    layer.trainable = True
model.load_weights('xception_weights_1.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_valid, y_valid))
model.save_weights('xception_weights_2.h5') 

Train on 19969 samples, validate on 4993 samples
Epoch 1/5
19969/19969 [==============================] - 147s 7ms/step - loss: 0.0436 - acc: 0.9847 - val_loss: 0.1206 - val_acc: 0.9696
Epoch 2/5
19969/19969 [==============================] - 142s 7ms/step - loss: 0.0149 - acc: 0.9956 - val_loss: 0.0207 - val_acc: 0.9942
Epoch 3/5
19969/19969 [==============================] - 142s 7ms/step - loss: 0.0048 - acc: 0.9987 - val_loss: 0.0217 - val_acc: 0.9958
Epoch 4/5
19969/19969 [==============================] - 142s 7ms/step - loss: 0.0016 - acc: 0.9995 - val_loss: 0.0236 - val_acc: 0.9946
Epoch 5/5
19969/19969 [==============================] - 142s 7ms/step - loss: 0.0033 - acc: 0.9991 - val_loss: 0.0209 - val_acc: 0.9948


In [14]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
df = pd.read_csv("sample_submission.csv")
for i, fname in enumerate(test_file):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('xception_predict_2.csv', index=None)
df.head(2)

12500/12500 [==============================] - 55s 4ms/step


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,0.995
1,2,0.995


在kaggle上的得分是0.04715

# 模型三
fine-tuning 锁前110层，放开110层之后的层参与训练

In [20]:
for layers in base_model.layers:
    layers.trainable = False
for layer in model.layers[110:]:
    layer.trainable = True
model.load_weights('xception_weights_2.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_valid, y_valid))

Train on 19969 samples, validate on 4993 samples
Epoch 1/5
19969/19969 [==============================] - 110s 6ms/step - loss: 0.0016 - acc: 0.9995 - val_loss: 0.0225 - val_acc: 0.9958
Epoch 2/5
19969/19969 [==============================] - 107s 5ms/step - loss: 2.7073e-04 - acc: 0.9999 - val_loss: 0.0245 - val_acc: 0.9946
Epoch 3/5
19969/19969 [==============================] - 107s 5ms/step - loss: 2.5106e-04 - acc: 0.9999 - val_loss: 0.0244 - val_acc: 0.9952
Epoch 4/5
19969/19969 [==============================] - 107s 5ms/step - loss: 3.3792e-04 - acc: 0.9999 - val_loss: 0.0234 - val_acc: 0.9954
Epoch 5/5
19969/19969 [==============================] - 107s 5ms/step - loss: 1.8252e-04 - acc: 0.9999 - val_loss: 0.0246 - val_acc: 0.9956


In [17]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
df = pd.read_csv("sample_submission.csv")
for i, fname in enumerate(test_file):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('xception_predict_3.csv', index=None)
df.head(2)

12500/12500 [==============================] - 51s 4ms/step


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,0.995
1,2,0.995


In [18]:
model.save_weights('xception_weights_3.h5') 

在kaggle上的得分是0.04663